In [10]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict

In [3]:
# import node embeddings
df = pd.read_csv(
    "./facebook_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-36_ind-36_k-10_ds-0.0.txt",
    sep="\t| ",
    header=None
)
df = df.rename(columns={0: 'node'})
df.set_index(['node'], inplace=True)

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [7]:
# import train
train = pd.read_csv("./facebook_train.csv", index_col=0)
# import test
test = pd.read_csv("./facebook_test.csv", index_col=0)

In [8]:
# node embeddings
display(df.head())
display(df.describe())

# ratings df
display(train.head())
display(train.describe())

# test df
display(test.head())
display(test.describe())

,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
node,,,,,,,,,,,,,,,,,,,,,
0,1.410215,1.055887,0.276106,0.337015,-0.855183,-2.079889,0.359319,-1.668620,0.876247,-1.597333,...,0.354289,-1.559046,-0.227659,0.615875,1.590270,-0.201614,-0.807248,0.872891,2.338391,0.422456
1,0.642537,0.210880,-0.079735,0.394628,-0.354652,-0.673384,-0.264542,-0.573317,0.387186,-0.928790,...,0.064035,-0.942434,-0.044016,0.357568,0.653752,-0.403103,-0.440986,0.245575,0.464131,0.173977
2,0.364444,0.483713,0.041530,-0.253252,-0.341561,-0.855875,0.262481,-0.731090,0.282645,-0.468689,...,0.558111,-0.349008,-0.063245,-0.019917,0.298720,0.008025,-0.167265,0.273740,0.909920,-0.209141
3,0.760110,0.369128,0.191771,0.325326,-0.209396,-0.604434,0.049080,-0.714295,0.193736,-0.875754,...,0.037971,-0.954742,-0.068425,0.373606,0.926925,-0.254449,-0.417411,0.377861,0.640826,0.327309
4,0.486993,0.028743,0.081050,-0.222652,-0.348967,-0.806270,0.705609,-0.529239,0.516686,-0.425610,...,0.268532,-0.495183,-0.013476,0.070995,0.500861,0.040166,0.071827,-0.117763,0.805720,-0.078896


,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
count,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,...,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000,3964.000000
mean,0.245962,0.234974,-0.118125,-0.069435,0.130555,-0.061437,0.135782,0.032777,-0.030051,0.159000,...,-0.045179,-0.055033,0.093342,0.198299,-0.009326,0.003912,-0.083582,0.338404,-0.135958,-0.183193
std,0.285687,0.536587,0.349763,0.284315,0.373223,0.393623,0.279270,0.570081,0.695714,0.558243,...,0.270464,0.555397,0.396318,0.625316,0.704484,0.415985,0.439189,0.330743,0.820394,0.626242
min,-0.692461,-0.683688,-2.198696,-1.892806,-0.855183,-2.079889,-0.787928,-1.668620,-1.995587,-1.597333,...,-0.955414,-1.706021,-0.641542,-0.842246,-2.712219,-1.082921,-1.936546,-0.436558,-3.588310,-1.661151
25%,0.045890,-0.118894,-0.274778,-0.250238,-0.115985,-0.191114,-0.023315,-0.245293,-0.274042,-0.058783,...,-0.217145,-0.276742,-0.104644,-0.125527,-0.058614,-0.209601,-0.204926,0.128835,-0.166092,-0.516550
50%,0.216832,0.047774,-0.047532,-0.041141,0.087722,-0.021151,0.123140,0.022771,-0.021171,0.086926,...,-0.056054,-0.050763,0.030199,0.071797,0.098033,-0.045417,0.018314,0.277037,0.001256,-0.290211
75%,0.364162,0.408460,0.114134,0.123863,0.323650,0.151224,0.264197,0.197018,0.114206,0.230962,...,0.097052,0.143825,0.202780,0.274561,0.329461,0.191753,0.175263,0.440702,0.196135,-0.043263
max,1.410215,2.177685,0.673434,0.971740,1.719087,1.372390,1.184074,2.493209,2.639332,2.432325,...,0.912800,3.097754,3.346251,2.757494,1.590270,2.287718,0.639313,1.988750,2.338391,2.369749


,u,v
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


,u,v
count,86999.000000,86999.000000
mean,1865.069449,2154.507420
std,910.682085,894.143924
min,0.000000,1.000000
25%,1160.000000,1601.000000
50%,1983.000000,2259.000000
75%,2423.000000,2631.000000
max,4031.000000,4038.000000


,u,v
0,0,276
1,9,21
2,21,281
3,25,141
4,26,9


,u,v
count,1169.000000,1169.000000
mean,2017.839179,2013.526091
std,823.568891,823.714657
min,0.000000,0.000000
25%,1413.000000,1388.000000
50%,2071.000000,2078.000000
75%,2603.000000,2601.000000
max,3980.000000,3993.000000


In [15]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.to_numpy().flatten()

def emb_distance(emb1, emb2):
    """computes the euclidean distance between two node embeddings"""
    return np.linalg.norm(emb1-emb2)

In [17]:
# get nx train graph
G_train = nx.from_pandas_edgelist(train, "u", "v")

In [18]:
# compute Top-N lists for test dataset
# for each testing row
results = {}
for i, test_row in test.iterrows():
    userId = test_row['u']
    user_emb = emb_from_row(df.iloc[userId])
    print(userId, " : ", user_emb)

    # get user's unwatched movies
    print("G_train.edges: ", G_train.edges)
    friends = list(G_train[userId])
    print("len(friends): ", len(friends))
    not_friends = df[~df.index.isin(friends+[userId])]
    print("len(not_friends): ", len(not_friends))

    # generate user's top_list from all not_friends
    distances = {}
    top_list = SortedDict()
    for movie_id, node_row in not_friends.iterrows():
        node_emb = emb_from_row(node_row)
        dist = emb_distance(user_emb, node_emb)
        top_list[dist] = movie_id
        distances[movie_id] = dist

    results[userId] = top_list, distances
    if userId % 10 == 0: print(f"{userId}/{len(test)}")

0  :  [ 1.4102145   1.0558867   0.2761059   0.33701485 -0.85518295 -2.0798888
  0.35931924 -1.6686199   0.87624687 -1.5973333   2.7001593  -0.50308746
 -1.8589901   0.5564184  -3.5769093  -1.0742759   0.1594586  -0.2903763
  1.8065327  -2.8263264  -1.2562433   1.8227433   0.02642364 -0.22394289
  2.3619406   2.283824    0.66626036  0.30710426  0.84068555  1.1530619
 -1.7353896  -1.013748    0.12268038  0.19129789 -0.3864959  -0.31557992
  0.01890535 -0.14656194  0.49439144  1.4672046   0.06936513  0.37190488
  3.750257    1.347548   -1.2072495   0.43932843  0.4021121   1.831826
  1.593666    0.6790248  -1.039928   -1.4345471  -0.27447048 -1.012325
  1.004277    2.1072185   0.41424385  1.4921106   0.38019958  0.3835879
 -0.8018281   5.688945    0.48816606  0.44731972  0.26997778 -0.3522951
 -0.8194676   2.4524949  -0.89037704 -2.6835642  -1.4060229   1.3790202
  2.3156533  -1.9220308   1.6188836  -1.3733168  -2.4106007   0.7638174
 -1.4703305   0.2489368  -1.4061484  -0.26856604  0.0878

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [NoneType]

In [24]:
N = 10

# counters
hits = 0
misses = 0

for userId in results.keys():
    (top_list, distances) = results[userId]

    # get Top-N friend suggestions
    top_N = top_list.values()[:N]

    # get user's test friend
    test_friend = test.query(f"u == {userId}")['v'][0]

    # evaluate
    hit = test_friend in top_N
    if hit:
        hits += 1
    else:
        misses += 1

    print(f"Friend suggestion {test_friend} {'found' if hit else 'not found'} in user {userId}'s Top-{N} list: ", top_N)


Movie 1017 not found in user 2's Top-50 list:  [3086, 4040, 3161, 3723, 2033, 2858, 2334, 3046, 3091, 3877, 2396, 2030, 2125, 2022, 4265, 3640, 4251, 3100, 3454, 3114, 3872, 2031, 3080, 2344, 3102, 3140, 1112, 4034, 2994, 3087, 3991, 3174, 4271, 3406, 1451, 2194, 3018, 3096, 3099, 4060, 3078, 2605, 2951, 2226, 3788, 3289, 4165, 1784, 4066, 2455]
Movie 1318 not found in user 3's Top-50 list:  [3081, 34166, 2951, 2035, 2380, 2416, 3096, 3022, 1914, 1252, 5018, 3752, 3829, 3121, 31816, 2588, 2605, 1273, 3978, 2022, 3263, 4612, 1709, 3097, 3143, 5277, 5380, 1742, 4997, 2013, 3112, 4999, 4737, 2487, 91866, 5994, 2081, 4604, 3141, 116122, 5023, 1420, 4301, 5867, 2686, 3017, 2575, 1917, 9880, 6159]
Movie 1034 not found in user 4's Top-50 list:  [6952, 8153, 2204, 41414, 5993, 5380, 67934, 51068, 7104, 115028, 100149, 107918, 28882, 7909, 116122, 4505, 8355, 80293, 1934, 3596, 59299, 7743, 139036, 3404, 104810, 46186, 5047, 55736, 8265, 37519, 5519, 37931, 9947, 55328, 8720, 80357, 8569, 8163,

In [25]:
print("hits: ", hits)
print("misses: ", misses)


hits:  21
misses:  623
